In [1]:
import zipfile

# Unzip the data
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [2]:
# !git clone https://github.com/bit-ml/dolos
!pip install streamlit pytorch-ignite tqdm face_recognition torchdata==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━

In [3]:
!git clone https://github.com/hilaryzen/DeepfakeEnsemble.git

Cloning into 'DeepfakeEnsemble'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 121 (delta 48), reused 78 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 22.01 MiB | 21.15 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [5]:
!mv DeepfakeEnsemble/dolos/* dolos/

In [6]:
!rm -rf DeepfakeEnsemble/

In [7]:
!wget https://sharing.speed.pub.ro/owncloud/index.php/s/MTSTY7tnKqn21h4/download -O dolos-models-patches.zip
!unzip dolos-models-patches.zip

--2025-02-02 05:08:00--  https://sharing.speed.pub.ro/owncloud/index.php/s/MTSTY7tnKqn21h4/download
Resolving sharing.speed.pub.ro (sharing.speed.pub.ro)... 141.85.160.7
Connecting to sharing.speed.pub.ro (sharing.speed.pub.ro)|141.85.160.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4365317 (4.2M) [application/zip]
Saving to: ‘dolos-models-patches.zip’

dolos-models-patche 100%[===================>]   4.16M  8.15MB/s    in 0.5s    

2025-02-02 05:08:04 (8.15 MB/s) - ‘dolos-models-patches.zip’ saved [4365317/4365317]

Archive:  dolos-models-patches.zip
  inflating: output/patch-forensics/full-supervision/setup-c-ffhq/best_model_49_neg-loss=-0.2153.pt  
  inflating: output/patch-forensics/full-supervision/setup-c/best_model_50_neg-loss=-0.0276.pt  
  inflating: output/patch-forensics/weak-supervision/setup-b-ffhq/best_model_100_neg-loss=-0.6733.pt  
  inflating: output/patch-forensics/weak-supervision/setup-b/best_model_99_neg-loss=-0.5980.pt  
  inflating:

In [29]:
import numpy as np
import os
import torch
import torch.nn.functional as F

from dolos.predict import (
    load_model,
    save_pred_as_img,
    get_predictions_path
)
from dolos.train_full_supervision import (
    get_mask_size
)
from dolos.train_weak_supervision import (
  # CONFIGS,
  TRANSFORM_IMAGE
)
from PIL import Image

num_images = 1500 # 1500 for test datasets, 2500 for train datasets

train_config_name = "setup-a"
print(CONFIGS)
train_config = CONFIGS[train_config_name]
device = "cuda" if torch.cuda.is_available() else "cpu"

method_name = "patch-forensics"
patch_model = load_model("weak", train_config_name, train_config, device)

mask_size = get_mask_size(train_config)
dolos_preds = np.zeros((num_images,))
dolos_file_num = np.zeros((num_images,))
dolos_actual = np.zeros((num_images,)) # Change depending on if images are real/df

path = 'data/individual/real/test/'
files = os.listdir(path)
for i in range(len(files)):
  image = Image.open(path + files[i]).convert('RGB')
  image = TRANSFORM_IMAGE["valid"](image)
  with torch.no_grad():
    image = image.to(device)
    mask_pred = patch_model(image.unsqueeze(0))

  mask_pred = F.softmax(mask_pred, dim=1)
  mask_pred = mask_pred[0, 1]
  mask_pred = mask_pred.detach().cpu().numpy()
  # Convert to 0 (real) and 1 (deepfake)
  dolos_preds[i] = 1 if np.mean(mask_pred) < 0.5 else 0

  file_num = files[i].split('.')[0][9:] # Change depending on test or train
  dolos_file_num[i] = int(file_num)

# Save image file numbers and predictions to .npz file
np.savez('dolos_iddd_test_real', dolos_preds=dolos_preds, dolos_actual=dolos_actual, dolos_file_num=dolos_file_num)


{'setup-b': {'load-image': <function load_image_0 at 0x7f35c57dd800>, 'last-layer': 'block2', 'frontend': None, 'output-transform': <function output_transform_expand at 0x7f35c57dda80>}, 'setup-b-ffhq': {'load-image': <function load_image_0 at 0x7f35c57dd800>, 'last-layer': 'block2', 'frontend': None, 'output-transform': <function output_transform_expand at 0x7f35c57dda80>}}


KeyError: 'setup-a'

In [13]:
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, recall_score, f1_score

data = np.load("dolos_iddd_test_df.npz")
print(accuracy_score(data["dolos_actual"], data["dolos_preds"]))
# print(len(np.unique(data["dolos_preds"])))
data.close()

0.0


In [15]:
real_data = np.load("dolos_iddd_test_real.npz")
df_data = np.load("dolos_iddd_test_df.npz")

preds = np.concatenate((real_data["dolos_preds"], df_data["dolos_preds"]))
actual = np.concatenate((real_data["dolos_actual"], df_data["dolos_actual"]))

print(accuracy_score(actual, preds))
print(precision_score(actual, preds))
print(recall_score(actual, preds))
print(f1_score(actual, preds))
print(average_precision_score(actual, preds))


real_data.close()
df_data.close()

0.5
0.0
0.0
0.0
0.5


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
